# Phi-4 Mini ONNX Paralel Fonksiyon Çağırma Eğitimi

Bu not defteri, Phi-4 Mini'nin ONNX Runtime GenAI ile paralel fonksiyon çağırma işlemini nasıl kullanacağınızı gösterir. Fonksiyon çağırma, modelin kullanıcı taleplerine göre harici araçları ve API'leri akıllıca çağırmasını sağlar.

## Genel Bakış

Bu eğitimde şunları öğreneceksiniz:
- Phi-4 Mini'yi ONNX Runtime GenAI ile kurma
- Uçuş ve otel rezervasyonu için fonksiyon şemaları tanımlama
- Yapılandırılmış çıktı için Lark grameri ile yönlendirilmiş üretim kullanma
- Karmaşık seyahat rezervasyonu senaryoları için paralel fonksiyon çağırma işlemlerini gerçekleştirme

## Ön Koşullar

Bu not defterini çalıştırmadan önce şunlara sahip olduğunuzdan emin olun:
- Phi-4 Mini ONNX modelini indirdiniz
- `onnxruntime-genai` paketini kurdunuz
- Fonksiyon çağırma kavramları hakkında temel bir anlayışınız var


## Adım 1: Gerekli Kütüphaneleri İçe Aktarma

Öncelikle, fonksiyon çağrısı uygulamamız için gerekli kütüphaneleri içe aktaracağız.


In [1]:
import json

In [2]:
import onnxruntime_genai as og

## Adım 2: Model Kurulumu ve Yapılandırma

Şimdi Phi-4 Mini ONNX modelini yapılandıracağız. Model dizininizi gerçek yolunuzla değiştirdiğinizden emin olun.


In [ ]:
# TODO: Replace with your actual Phi-4 Mini ONNX model path
# Download from: https://huggingface.co/microsoft/Phi-4-mini-onnx
path = 'Your phi-4-mini-onnx path'  # Update this path!

In [4]:
config = og.Config(path)

In [5]:
model = og.Model(config)

In [6]:
tokenizer = og.Tokenizer(model)
tokenizer_stream = tokenizer.create_stream()

## Adım 3: Üretim Parametrelerini Yapılandırma

Modelin çıktı davranışını kontrol etmek için üretim parametrelerini ayarlayın. Bu ayarlar, işlev çağrıları için belirli ve odaklanmış yanıtlar sağlar.


In [7]:
# Configure generation parameters for deterministic function calling
search_options = {}
search_options['max_length'] = 4096      # Maximum tokens to generate
search_options['temperature'] = 0.00001  # Very low temperature for deterministic output
search_options['top_p'] = 1.0            # Nucleus sampling parameter
search_options['do_sample'] = False       # Disable sampling for consistent results

## Adım 4: Mevcut Fonksiyonları Tanımlayın

Burada, yapay zeka asistanımızın çağırabileceği fonksiyonları tanımlıyoruz. Bu örnekte, seyahatle ilgili iki fonksiyonumuz var:
1. **booking_flight_tickets**: Havaalanları arasında uçuş rezervasyonu yapmak için
2. **booking_hotels**: Otel konaklamaları rezervasyonu yapmak için

Fonksiyon tanımları, OpenAI'nin fonksiyon çağırma şema formatını takip eder.


In [8]:
tool_list = '[{"name": "booking_flight_tickets", "description": "booking flights", "parameters": {"origin_airport_code": {"description": "The name of Departure airport code", "type": "string"}, "destination_airport_code": {"description": "The name of Destination airport code", "type": "string"}, "departure_date": {"description": "The date of outbound flight", "type": "string"}, "return_date": {"description": "The date of return flight", "type": "string"}}}, {"name": "booking_hotels", "description": "booking hotel", "parameters": {"destination": {"description": "The name of the city", "type": "string"}, "check_in_date": {"description": "The date of check in", "type": "string"}, "checkout_date": {"description": "The date of check out", "type": "string"}}}]'

## Adım 5: Dilbilgisi Oluşturma Yardımcı Fonksiyonları

Bu yardımcı fonksiyonlar, fonksiyon tanımlarımızı ONNX Runtime GenAI tarafından yönlendirilmiş üretim için kullanılan Lark dilbilgisi formatına dönüştürür. Bu, modelin doğru JSON formatında geçerli fonksiyon çağrıları üretmesini sağlar.


In [9]:
def get_lark_grammar(input_tools):
    tools_list = get_tools_list(input_tools)
    prompt_tool_input = create_prompt_tool_input(tools_list)
    if len(tools_list) == 1:
        # output = ("start: TEXT | fun_call\n" "TEXT: /[^{](.|\\n)*/\n" " fun_call: <|tool_call|> %json " + json.dumps(tools_list[0]))
        output = ("start: TEXT | fun_call\n" "TEXT: /[^{](.|\\n)*/\n" " fun_call: <|tool_call|> %json " + json.dumps(convert_tool_to_grammar_input(tools_list[0])))
        return prompt_tool_input, output
    else:
        return prompt_tool_input, "start: TEXT | fun_call \n TEXT: /[^{](.|\n)*/ \n fun_call: <|tool_call|> %json {\"anyOf\": [" + ','.join([json.dumps(tool) for tool in tools_list]) + "]}"


In [10]:
def get_tools_list(input_tools):
    # input_tools format: '[{"name": "fn1", "description": "fn details", "parameters": {"p1": {"description": "details", "type": "string"}}},
    # {"fn2": 2},{"fn3": 3}]'
    tools_list = []
    try:
        tools_list = json.loads(input_tools)
    except json.JSONDecodeError:
        raise ValueError("Invalid JSON format for tools list, expected format: '[{\"name\": \"fn1\"},{\"name\": \"fn2\"}]'")
    if len(tools_list) == 0:
        raise ValueError("Tools list cannot be empty")
    return tools_list

In [11]:
def create_prompt_tool_input(tools_list):
    tool_input = str(tools_list[0])
    for tool in tools_list[1:]:
        tool_input += ',' + str(tool)
    return tool_input

In [12]:
def convert_tool_to_grammar_input(tool):
    param_props = {}
    required_params = []
    for param_name, param_info in tool.get("parameters", {}).items():
        param_props[param_name] = {
            "type": param_info.get("type", "string"),
            "description": param_info.get("description", "")
        }
        required_params.append(param_name)
    output_schema = {
        "description": tool.get('description', ''),
        "type": "object",
        "required": ["name", "parameters"],
        "additionalProperties": False,
        "properties": {
            "name": { "const": tool["name"] },
            "parameters": {
                "type": "object",
                "properties": param_props,
                "required": required_params,
                "additionalProperties": False
            }
        }
    }
    if len(param_props) == 0:
        output_schema["required"] = ["name"]
    return output_schema

In [13]:
get_lark_grammar(tool_list)

("{'name': 'booking_flight_tickets', 'description': 'booking flights', 'parameters': {'origin_airport_code': {'description': 'The name of Departure airport code', 'type': 'string'}, 'destination_airport_code': {'description': 'The name of Destination airport code', 'type': 'string'}, 'departure_date': {'description': 'The date of outbound flight', 'type': 'string'}, 'return_date': {'description': 'The date of return flight', 'type': 'string'}}},{'name': 'booking_hotels', 'description': 'booking hotel', 'parameters': {'destination': {'description': 'The name of the city', 'type': 'string'}, 'check_in_date': {'description': 'The date of check in', 'type': 'string'}, 'checkout_date': {'description': 'The date of check out', 'type': 'string'}}}",
 'start: TEXT | fun_call \n TEXT: /[^{](.|\n)*/ \n fun_call: <|tool_call|> %json {"anyOf": [{"name": "booking_flight_tickets", "description": "booking flights", "parameters": {"origin_airport_code": {"description": "The name of Departure airport c

## Adım 6: Dilbilgisi Oluşturmayı Test Et

Dilbilgisi oluşturma fonksiyonlarımızı test edelim ve araç tanımlarımızı doğru formata nasıl dönüştürdüklerini görelim.


In [14]:
prompt_tool_input, guidance_input = get_lark_grammar(tool_list)

## Adım 7: Sistem İsteğini ve Üreteci Hazırlayın

Şimdi modelin kullanılabilir araçlar hakkında bilgi almasını sağlayan sistem isteğini oluşturacağız ve yönlendirilmiş üretim parametreleriyle üreteci ayarlayacağız.


In [15]:
# Define the system prompt that introduces the assistant and its capabilities
system_prompt = "You are a helpful assistant with these tools."

In [16]:
# Format the system message with tools information
messages = f"""[{{"role": "system", "content": "{system_prompt}", "tools": "{prompt_tool_input}"}}]"""

In [17]:
# Apply chat template to format the system prompt properly
tokenizer_input_system_prompt = tokenizer.apply_chat_template(messages=messages, add_generation_prompt=False)

In [18]:
tokenizer_input_system_prompt

"<|system|>You are a helpful assistant with these tools.<|tool|>{'name': 'booking_flight_tickets', 'description': 'booking flights', 'parameters': {'origin_airport_code': {'description': 'The name of Departure airport code', 'type': 'string'}, 'destination_airport_code': {'description': 'The name of Destination airport code', 'type': 'string'}, 'departure_date': {'description': 'The date of outbound flight', 'type': 'string'}, 'return_date': {'description': 'The date of return flight', 'type': 'string'}}},{'name': 'booking_hotels', 'description': 'booking hotel', 'parameters': {'destination': {'description': 'The name of the city', 'type': 'string'}, 'check_in_date': {'description': 'The date of check in', 'type': 'string'}, 'checkout_date': {'description': 'The date of check out', 'type': 'string'}}}<|/tool|><|end|><|endoftext|>"

In [19]:
input_tokens = tokenizer.encode(tokenizer_input_system_prompt)

In [20]:
input_tokens = input_tokens[:-1]

In [21]:
system_prompt_length = len(input_tokens)

## Adım 8: Yönlendirilmiş Üretim ile Üreteci Başlat

Şimdi yapılandırılmış parametrelerimizle üreteci oluşturacağız ve yönlendirilmiş üretim için Lark dilbilgisini uygulayacağız.


In [22]:
# Create generator parameters and apply search options
params = og.GeneratorParams(model)
params.set_search_options(**search_options)

In [23]:
# Apply Lark grammar for guided generation to ensure valid function call format
params.set_guidance("lark_grammar", guidance_input)

In [24]:
generator = og.Generator(model, params)

In [25]:
generator.append_tokens(input_tokens)

## Adım 9: Paralel Fonksiyon Çağrısını Test Etme

Şimdi, birden fazla fonksiyon çağrısı gerektiren karmaşık bir seyahat rezervasyonu senaryosu ile kurulumumuzu test edelim.


In [26]:
# Complex travel booking request that requires both flight and hotel booking
text = "book flight ticket from Beijing to Paris(using airport code) in 2025-12-04 to 2025-12-10 , then book hotel from 2025-12-04 to 2025-12-10 in Paris"

In [27]:
# Format user message and apply chat template
messages = f"""[{{"role": "user", "content": "{text}"}}]"""

# Apply Chat Template for user input
user_prompt = tokenizer.apply_chat_template(messages=messages, add_generation_prompt=True)
input_tokens = tokenizer.encode(user_prompt)
generator.append_tokens(input_tokens)

In [28]:
user_prompt

'<|user|>book flight ticket from Beijing to Paris(using airport code) in 2025-12-04 to 2025-12-10 , then book hotel from 2025-12-04 to 2025-12-10 in Paris<|end|><|assistant|>'

### Fonksiyon Çağrıları Oluşturma

Model artık kullanıcı isteğine göre yapılandırılmış fonksiyon çağrıları oluşturacak. Yönlendirilmiş üretim sayesinde, çıktı doğrudan çalıştırılabilir geçerli JSON formatında olacak.

**Beklenen Çıktı**: Model şu fonksiyon çağrılarını oluşturmalıdır:
1. Beijing (PEK) ile Paris (CDG) detayları için `booking_flight_tickets`
2. Paris konaklama detayları için `booking_hotels`

Aşağıdaki hücreyi çalıştırarak canlı üretimi görebilirsiniz:


In [29]:
# Generate tokens one by one and stream the output
while not generator.is_done():
    generator.generate_next_token()
    new_token = generator.get_next_tokens()[0]
    print(tokenizer_stream.decode(new_token), end='', flush=True)

[{"name": "booking_flight_tickets", "arguments": {"origin_airport_code": "PEKK", "destination_airport_code": "CDG", "departure_date": "2025-12-04", "return_date": "2025-12-10"}}, {"name": "booking_hotels", "arguments": {"destination": "Paris", "check_in_date": "2025-12-04", "checkout_date": "2025-12-10"}}]

## Sonuç

🎉 **Tebrikler!** Phi-4 Mini ile ONNX Runtime GenAI kullanarak paralel fonksiyon çağrısını başarıyla uyguladınız.

### Öğrendikleriniz:

1. **Model Kurulumu**: Phi-4 Mini'yi ONNX Runtime GenAI ile nasıl yapılandıracağınızı
2. **Fonksiyon Tanımı**: AI fonksiyon çağrıları için fonksiyon şemalarını nasıl tanımlayacağınızı
3. **Yönlendirilmiş Üretim**: Yapılandırılmış çıktı üretimi için Lark dilbilgisini nasıl kullanacağınızı
4. **Paralel Fonksiyon Çağrıları**: Birden fazla fonksiyon çağrısı gerektiren karmaşık talepleri nasıl yöneteceğinizi

### Temel Faydalar:

- ✅ **Yapılandırılmış Çıktı**: Yönlendirilmiş üretim, geçerli JSON fonksiyon çağrılarını garanti eder
- ✅ **Paralel İşleme**: Tek bir talepte birden fazla fonksiyon çağrısını yönetme
- ✅ **Yüksek Performans**: ONNX Runtime optimize edilmiş çıkarım sağlar
- ✅ **Esnek Şema**: Fonksiyon tanımlarını kolayca ekleme veya değiştirme

### Kaynaklar:

- [Phi-4 Mini Belgeleri](https://huggingface.co/microsoft/Phi-4-mini-onnx)
- [ONNX Runtime GenAI Belgeleri](https://onnxruntime.ai/docs/genai/)
- [Fonksiyon Çağrısı En İyi Uygulamaları](https://platform.openai.com/docs/guides/function-calling)



---

**Feragatname**:  
Bu belge, AI çeviri hizmeti [Co-op Translator](https://github.com/Azure/co-op-translator) kullanılarak çevrilmiştir. Doğruluğu sağlamak için çaba göstersek de, otomatik çevirilerin hata veya yanlışlık içerebileceğini lütfen unutmayın. Belgenin orijinal dili, yetkili kaynak olarak kabul edilmelidir. Kritik bilgiler için profesyonel insan çevirisi önerilir. Bu çevirinin kullanımından kaynaklanan yanlış anlamalar veya yanlış yorumlamalar için sorumluluk kabul etmiyoruz.
